In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append('../')

from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [ ]:
from database.models import KabuDatabase, KabuPrice
from get_kabuka import GetKabuka

DB = KabuDatabase()
KP = KabuPrice()
GK = GetKabuka()

In [ ]:
df_stock = pd.DataFrame(np.array(DB.get_data('all_code')),columns = ['id','code','name'])
codes = list(df_stock['code'])
codes = codes[:10]

def multi_process(code):
    try:
        df_got = GK.get_latest_data(code).sort_values('日付')
        KP.update_data(df_got,code)
        print('OK', code, df_got.iloc[0,0])
    except:
        print('NG', code, df_got.iloc[0,0])
        pass

In [ ]:
#single process
for code in codes:
    multi_process(code)

In [ ]:
#multi process
with ProcessPoolExecutor(max_workers = 6) as p:
    p.map(multi_process,codes)

In [ ]:
from datetime import datetime
import time
def task(num):
    start = datetime.now()
    time.sleep(2)
    print('number',num)
    return (num, start, datetime.now())

### sample

In [1]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import time
from tqdm import tqdm

In [37]:
def task(_n):
    s = 0
    for i in range(1,_n+1):
        s+=i
        time.sleep(0.1)
    return s

In [38]:
ns = list(np.arange(1,11))

In [64]:
def task_from_to(_args):
    from_, to_ = _args[0], _args[1]
    s = 0
    for i in range(from_, to_):
        s+=i
        time.sleep(0.1)
    return s

In [65]:
args = [[i+3,(i+1)*3] for i in range(10)]

### single process

In [39]:
start = time.time()

sms_single = []
for n in tqdm(ns):
    sms_single.append(task(n))
    
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3)))

100%|██████████| 10/10 [00:05<00:00,  1.28it/s]

処理時間:5.533s


### multithreading

In [40]:
start = time.time()

with ThreadPoolExecutor(6) as e:
    ret = e.map(task, ns)
    sms_multi = [r for r in ret]
    
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3)))   

処理時間:1.405s


In [47]:
start = time.time()

with ThreadPoolExecutor(12) as e:
    ret = e.map(task, ns)
    sms_multi = [r for r in tqdm(ret)]
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3))) 

10it [00:01,  9.92it/s]

処理時間:1.012s


### multiprocess

In [45]:
start = time.time()

with ProcessPoolExecutor(6) as e:
    ret = e.map(task, ns)
sms_multi = [r for r in ret]
    
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3)))   

処理時間:1.422s


In [67]:
start = time.time()

with ProcessPoolExecutor(12) as e:
    ret = e.map(task, ns)
    sms_multi = [r for r in ret]
    
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3)))  

TypeError: task() missing 1 required positional argument: 'to_'

In [68]:
start = time.time()

with ProcessPoolExecutor(12) as e:
    ret = e.map(task_from_to, args)
sms_multi = [r for r in ret]
    
end = time.time()
delta = end - start
print('処理時間:{}s'.format(round(delta,3)))  

処理時間:1.833s
